## Intialize

In [1]:
# import necessary libraries
import pandas as pd
from collections import Counter 
from datetime import datetime

## Functions

In [2]:
def getDistrict(refData, street, month, day):
    
    # exact match
    districts = refData[(refData.STREET == street) & (refData.MONTH == month) & (refData.DAY == day)].DISTRICT
    
    if len(set(districts)) == 1:
        bestMatch = districts.iloc[0]
    
    elif len(set(districts)) == 0:
             newMatches = refData[(refData.STREET == street) & (refData.MONTH == month)].DISTRICT
             
             if len(set(newMatches)) == 1:
                 bestMatch = newMatches.iloc[0]
             
             elif len(set(newMatches)) == 0:
                 finalMatches = refData[(refData.STREET == street)].DISTRICT
             
                 if len(set(finalMatches)) == 1:
                     bestMatch = finalMatches.iloc[0]

                 elif len(set(finalMatches)) == 0:
                     bestMatch = 'Unknown District'

                 else:
                     finalFreq = Counter(finalMatches)
                     bestMatch = finalFreq.most_common(1)[0][0]
             
             else:
                 newFreq = Counter(newMatches)
                 bestMatch = newFreq.most_common(1)[0][0]
             
    else:
        freq = Counter(districts)
        bestMatch = freq.most_common(1)[0][0]
        
    
    return bestMatch

In [3]:
def getDate(dateStr):
    
    date_obj = datetime.strptime(dateStr, '%Y-%m-%d %H:%M:%S')

    # Extract the year, month, and day
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    hour = date_obj.hour

    return (year, month, day, hour)

## Preparing Crimes Data

In [4]:
# load data
crimes = pd.read_csv('crimes.csv', low_memory=False)

In [5]:
# get the day of the month for each crime
crimes['DAY'] = crimes['OCCURRED_ON_DATE'].apply(lambda x: getDate(x)[2])

In [6]:
# replace weekday categorical variables with 2 binary variables
days = {'Sunday': [0, 1], 'Monday': [1, 0], 'Tuesday': [2, 0], 'Wednesday': [3, 0], 'Thursday': [4, 0],
       'Friday': [5, 1], 'Saturday': [6, 1]}

crimes['WEEKDAY'] = crimes['DAY_OF_WEEK'].apply(lambda x: days[x][0])
crimes['WEEKEND'] = crimes['DAY_OF_WEEK'].apply(lambda x: days[x][1])

In [7]:
# do one hot encoding for types of crimes
oneHotCrime = pd.get_dummies(crimes['OFFENSE_DESCRIPTION'])

In [8]:
# determine which columns i need
cCols = [
 'DISTRICT',
 'SHOOTING',
 'MONTH',
 'WEEKDAY',
 'WEEKEND',
 'HOUR',
 'DAY']

In [9]:
# create sub-dfs with only essential columns
finalCrimes = crimes[cCols].join(oneHotCrime)

In [10]:
# create comprehensive dataframe
crCompCols = ['OFFENSE_DESCRIPTION'] + cCols
crCompCols.remove('WEEKDAY')
crimesComp = crimes[crCompCols]

## Preparing Field Contact Data

In [11]:
# load data
fieldC = pd.read_csv('fieldContact.csv')
fieldCN = pd.read_csv('fieldContactName.csv')

In [12]:
# get time variables for the field contact per officer incident
fieldC['year'] = fieldC['contact_date'].apply(lambda x: getDate(x)[0])
fieldC['month'] = fieldC['contact_date'].apply(lambda x: getDate(x)[1])
fieldC['day'] = fieldC['contact_date'].apply(lambda x: getDate(x)[2])
fieldC['hour'] = fieldC['contact_date'].apply(lambda x: getDate(x)[3])

In [13]:
# get districts from streets
fieldC = fieldC.dropna(subset=['street'])
fieldC['district'] = fieldC.apply(lambda x: getDistrict(crimes, x['street'], x['month'], x['day']), axis=1)

In [14]:
# add contact hour to fieldCN data
fieldCN['properDate'] = fieldCN['contact_date'].apply(lambda x: x.split('.')[0])
fieldCN['contactHour'] = fieldCN['properDate'].apply(lambda x: getDate(x)[3])

In [15]:
# create one hot encoding dataframes for categorical variables of fieldCN
oneHotSex = pd.get_dummies(fieldCN['sex'])
oneHotRace = pd.get_dummies(fieldCN['race'])
oneHotBuild = pd.get_dummies(fieldCN['build'])
oneHotHair = pd.get_dummies(fieldCN['hair_style'])
oneHotEth = pd.get_dummies(fieldCN['ethnicity'])
oneHotSkin = pd.get_dummies(fieldCN['skin_tone'])

In [16]:
# change columns names so there are no duplicate columns
oneHotSex = oneHotSex.rename(columns={'Unknown': 'Unknown Sex'})
oneHotRace = oneHotRace.rename(columns={'Unknown': 'Unknown Race', 'Other':'OtherRace'})
oneHotBuild = oneHotBuild.rename(columns={'Unknown': 'Unknown Build'})
oneHotHair = oneHotHair.rename(columns={'Unknown': 'Unknown Hair', 'Other':'OtherHair'})
oneHotEth = oneHotEth.rename(columns={'Unknown': 'Unknown Ethnicity'})
oneHotSkin = oneHotSkin.rename(columns={'Unknown': 'Unknown Skin Color', 'OTHER':'Other Skin Color', 'Black': 'Black Skin Color',
                                       'Medium': 'Medium Skin Color'})


In [17]:
# create one hot encodings for categorical variabled of fieldC
oneHotCirc = pd.get_dummies(fieldC['circumstance'])
oneHotBasis = pd.get_dummies(fieldC['basis'])
oneHotVehMod = pd.get_dummies(fieldC['vehicle_model'])
oneHotVehSty = pd.get_dummies(fieldC['vehicle_style'])
oneHotVehTyp = pd.get_dummies(fieldC['vehicle_type'])
oneHotSit = pd.get_dummies(fieldC['weather'])

In [18]:
# change columns names so there are no duplicate columns
oneHotVehMod = oneHotVehMod.rename(columns={'OTHER': 'Other Vehicle Model'})
oneHotVehSty = oneHotVehSty.rename(columns={'Other': 'Other Vehicle Style', 'Van': 'Van (Style)'})
oneHotSit = oneHotSit.drop(columns=['Encounter', 'Intel', 'Probable Cause', 'Reasonable Suspicion '])

In [19]:
# determine columns I need from each df
fcnCols = [
 'fc_num',
 'age',
 'was_frisked',
 'contactHour'
]

fcCols = ['fc_num',
          'contact_officer_name',
          'supervisor_name',
         'city',
         'zip',
         'stop_duration',
         'month',
         'day',
         'hour',
         'district',
         'contact_reason'
         ]

In [20]:
# create sub-df with only essential columns
finalFC = fieldC[fcCols]
finalFCN = fieldCN[fcnCols].merge(finalFC, on='fc_num', how='left')

In [21]:
# join one hot encoding dfs and main df together
fieldOH = finalFCN.join(oneHotSex)
fieldOH = fieldOH.join(oneHotRace)
fieldOH = fieldOH.join(oneHotBuild)
fieldOH = fieldOH.join(oneHotHair)
fieldOH = fieldOH.join(oneHotEth)
fieldOH = fieldOH.join(oneHotSkin)

fieldOH = fieldOH.join(oneHotCirc)

fieldOH = fieldOH.join(oneHotBasis)

fieldOH = fieldOH.join(oneHotVehMod)

fieldOH = fieldOH.join(oneHotVehSty)

fieldOH = fieldOH.join(oneHotVehTyp)

fieldOH = fieldOH.join(oneHotSit)

In [22]:
# create comprehensive dataframe
fcnCompCols = ['sex', 'race', 'build', 'hair_style', 'ethnicity', 'skin_tone'] + fcnCols
fcCompCols = ['circumstance', 'basis', 'vehicle_model', 'vehicle_style', 'vehicle_year', 'vehicle_type', 
              'weather', 'vehicle_state', ] + fcCols

fcnComp = fieldCN[fcnCompCols]
fcComp = fieldC[fcCompCols]
fieldComp = fcnComp.merge(fcComp, on='fc_num', how='left')

## Preparing Shooting Data

In [23]:
# load data
shooting = pd.read_csv('shootings.csv')
shots = pd.read_csv('shotsFired.csv')

In [24]:
# only get 2022 data for shootings
shooting = shooting[shooting['shooting_date'].str.contains('2022')]

In [25]:
# create new column that uses binary values for if there were multiple victims
MV = {'f': 0, 't': 1}
shooting['victim_plural'] = shooting['multi_victim'].apply(lambda x: MV[x])

In [26]:
# get time variables for shooting occurence
shooting['date'] = shooting['shooting_date'].apply(lambda x: x.split('+')[0])
shooting['year'] = shooting['date'].apply(lambda x: getDate(x)[0])
shooting['month'] = shooting['date'].apply(lambda x: getDate(x)[1])
shooting['day'] = shooting['date'].apply(lambda x: getDate(x)[2])
shooting['hour'] = shooting['date'].apply(lambda x: getDate(x)[3])

In [27]:
# create one hot encoding dfs for categorical variables
oneHotDeath = pd.get_dummies(shooting['shooting_type_v2'])
oneHotGen = pd.get_dummies(shooting['victim_gender'])
oneHotVRace = pd.get_dummies(shooting['victim_race'])
oneHotVEth = pd.get_dummies(shooting['victim_ethnicity_NIBRS'])

In [28]:
# change columns names so there are no duplicate columns
oneHotDeath = oneHotDeath.drop(columns=['Non-Fatal'])
oneHotVRace = oneHotVRace.rename(columns={'Unknown': 'Unknown Race'})
oneHotVEth = oneHotVEth.rename(columns={'Unknown': 'Unknown Ethnicity'})

In [29]:
# determine columns I need from each df
sCols = [
 'district',
 'victim_plural',
 'month',
 'day',
 'hour']

In [30]:
# create sub-df with only essential columns
finalSh = shooting[sCols]

In [31]:
# join one hot encoding dfs and main df together
shootingOH = finalSh.join(oneHotDeath)
shootingOH = shootingOH.join(oneHotGen)
shootingOH = shootingOH.join(oneHotVRace)
shootingOH = shootingOH.join(oneHotVEth)

In [32]:
# create comprehensive dataframe
shCompCols = ['shooting_type_v2', 'victim_gender', 'victim_race', 'victim_ethnicity_NIBRS'] + sCols

shootingComp = shooting[shCompCols]

## Save One Hot Encoding Data

In [33]:
# make all df cols lowercase
finalCrimes.columns = finalCrimes.columns.str.lower()
shootingOH.columns = shootingOH.columns.str.lower()
fieldOH.columns = fieldOH.columns.str.lower()

In [34]:
# save all to csvs
finalCrimes.to_csv('crimesOH.csv', index=False)
shootingOH.to_csv('shootingOH.csv', index=False)
fieldOH.to_csv('fieldOH.csv', index=False)

## Save Clean Comprehensive Data

In [35]:
# make all df cols lowercase
shootingComp.columns = shootingComp.columns.str.lower()
fieldComp.columns = fieldComp.columns.str.lower()
crimesComp.columns = crimesComp.columns.str.lower()

In [36]:
shootingComp.to_csv('shootingComp.csv', index=False)
fieldComp.to_csv('fieldComp.csv', index=False)
crimesComp.to_csv('crimesComp.csv', index=False)

## Save Summary Data

In [37]:
shooting_count = shootingComp.groupby('district').count().reset_index()
field_count = fieldComp.groupby('district').count().reset_index()
crimes_count = crimesComp.groupby('district').count().reset_index()

In [38]:
summary = pd.DataFrame()
summary['district'] = crimes_count['district']
summary['shootings'] = shooting_count[['shooting_type_v2']]
summary['crimes'] = crimes_count[['hour']]
summary['fieldInt'] = field_count[['hour']]

In [39]:
summary.to_csv('summary.csv', index=False)